# Exploratory Data Analysis 

## Performed on the energy network dataset, to verify outliers, distribution and meaningful graphs

In [1]:
# pip install sweetviz
# conda install -c conda-forge lux-api

# Specific libraries
# import sweetviz as sv
import autoviz
from autoviz.AutoViz_Class import AutoViz_Class
# import lux

# General libraries
import sys, os
from os import system
import warnings

import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime as dt
from datetime import date
from datetime import timedelta

Imported AutoViz_Class version: 0.0.81. Call using:
    from autoviz.AutoViz_Class import AutoViz_Class
    AV = AutoViz_Class()
    AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=0,
                            lowess=False,chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30)
Note: verbose=0 or 1 generates charts and displays them in your local Jupyter notebook.
      verbose=2 saves plots in your local machine under AutoViz_Plots directory and does not display charts.


### Part 0: Integrating the data

In [2]:
# Constants
values_column_names = ["time", "branch" , "organization", "substation", "transformer_code", "App SW", 
                        "V_L1", "I_L1", "W_L1", "QL_L1", "QC_L1","cos_L1", "angle_L1",
                        "V_L2", "I_L2", "W_L2", "QL_L2", "QC_L2","cos_L2", "angle_L2",
                        "V_L3", "I_L3", "W_L3", "QL_L3", "QC_L3","cos_L3", "angle_L3",
                        "temp_amb",
                        "aplus_L1", "aminus_L1", "RplusL_L1", "RminusL_L1", "RplusC_L1", "RminusC_L1", 
                        "aplus_L2", "aminus_L2", "RplusL_L2", "RminusL_L2", "RplusC_L2", "RminusC_L2",
                        "aplus_L3", "aminus_L3", "RplusL_L3", "RminusL_L3", "RplusC_L3", "RminusC_L3"]

script_path = os.getcwd()
data = pd.read_csv('../DATA/LVSM_Def.csv',  sep = ';', header=0, names=values_column_names)

# Cleaning data table
data = data.drop(["aminus_L1", "RminusL_L1", "RplusC_L1", 
                  "aminus_L2", "RminusL_L2", "RplusC_L2",
                  "aminus_L3", "RminusL_L3", "RplusC_L3"], axis=1)

data = data.reset_index(drop = True)

# Change column types to appropiate
data = data.astype({"time": str, "branch": str , "organization": str, "substation": str, "transformer_code": str, "App SW": str})

data[["V_L1", "I_L1", "W_L1", "QL_L1", "QC_L1","cos_L1", "angle_L1",
      "V_L2", "I_L2", "W_L2", "QL_L2", "QC_L2","cos_L2", "angle_L2",
      "V_L3", "I_L3", "W_L3", "QL_L3", "QC_L3","cos_L3", "angle_L3",
      "temp_amb"]] = data[["V_L1", "I_L1", "W_L1", "QL_L1", "QC_L1","cos_L1", "angle_L1",
                           "V_L2", "I_L2", "W_L2", "QL_L2", "QC_L2","cos_L2", "angle_L2",
                           "V_L3", "I_L3", "W_L3", "QL_L3", "QC_L3","cos_L3", "angle_L3",
                           "temp_amb"]].astype(float)


### Deal with the "24:00" problem. Adapt BOTH the hour and the day.
# Get the indexes and replace hour
for i, date in enumerate(data['time']):
    if date.split()[1].split(':')[0] == '24':
        data.loc[i, 'time'] = data.loc[i, 'time'].replace("24:00","00:00")
        data.loc[i, 'time'] = pd.to_datetime(data.loc[i, 'time'], format = '%Y-%m-%d %H:%M') + timedelta(days = 1)

# Update the format
data['time'] = pd.to_datetime(data['time'], format = '%Y-%m-%d %H:%M:%S')


In [3]:
# Copy of the dataframe to split date and hour
data_new = data.copy(deep=True)

In [4]:
### Preformat

# Split the time column into date and hour columns, for diagram's input preparation
data_new['date'] = (data_new['time']).dt.date
data_new['hour'] = (data_new['time']).dt.time

# Delete the old time column
data_new = data_new.drop(["time"], axis=1)

# Put both columns at the start
data_new = pd.concat([data_new['hour'], data_new.drop('hour',axis=1)], axis=1)
data_new = pd.concat([data_new['date'], data_new.drop('date',axis=1)], axis=1)

# Cleaning NA values
if data_new.isna().sum().sum() < .10 * len(data_new): 
    print ("Cleaning NA values from dataset")
    data_new = data_new.dropna()
else:
    raise Exception("Careful! Deleting NaN values would cut most of the dataset")

# Remove duplicates
if data.duplicated().sum() < .10 * len(data_new): 
    print ("Cleaning duplicate values from dataset")
    data_new = data_new.drop_duplicates(subset=['date', 'hour', 'substation', 'App SW'])
else:
    raise Exception("Careful! Deleting duplicated values would cut most of the dataset")




Cleaning NA values from dataset
Cleaning duplicate values from dataset


In [5]:
warnings.filterwarnings('ignore')
data_new.head()

,date,hour,branch,organization,substation,transformer_code,App SW,V_L1,I_L1,W_L1,...,temp_amb,aplus_L1,RplusL_L1,RminusC_L1,aplus_L2,RplusL_L2,RminusC_L2,aplus_L3,RplusL_L3,RminusC_L3
0,2019-06-16,01:00:00,AE,SZZ,S201,TR1,003F,234.0,65.0,14964.0,...,30.0,16082.0,1983.0,0.0,16736.0,1620.0,0.0,23015.0,2179.0,0.0
1,2019-06-16,02:00:00,AE,SZZ,S201,TR1,003F,233.0,57.0,13091.0,...,29.0,14342.0,1441.0,0.0,14545.0,1057.0,28.0,23764.0,2906.0,0.0
2,2019-06-16,03:00:00,AE,SZZ,S201,TR1,003F,236.0,55.0,12847.0,...,29.0,13543.0,1381.0,0.0,14073.0,1141.0,0.0,22147.0,2942.0,0.0
3,2019-06-16,04:00:00,AE,SZZ,S201,TR1,003F,234.0,135.0,30517.0,...,29.0,20757.0,2954.0,0.0,22059.0,2021.0,0.0,27317.0,3701.0,0.0
4,2019-06-16,05:00:00,AE,SZZ,S201,TR1,003F,235.0,102.0,23069.0,...,29.0,29753.0,5054.0,0.0,31259.0,3121.0,2.0,33013.0,3778.0,0.0


In [6]:
# Prepare the train and test dataset
msk = np.random.rand(len(data_new)) < 0.98

df_train = data_new[msk]
df_test = data_new[~msk]

### Part 0: Automating EDA - Pandas methods

In [7]:
data_new.describe()

,V_L1,I_L1,W_L1,QL_L1,QC_L1,cos_L1,angle_L1,V_L2,I_L2,W_L2,...,temp_amb,aplus_L1,RplusL_L1,RminusC_L1,aplus_L2,RplusL_L2,RminusC_L2,aplus_L3,RplusL_L3,RminusC_L3
count,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,...,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000,164433.000000
mean,233.955271,229.200878,51290.507058,4815.429981,-1102.215942,0.422519,34.157633,234.206260,228.577481,51193.397293,...,27.055743,52862.287357,4824.976988,1717.213169,52780.178450,4436.669744,1927.236802,52245.420487,4791.569235,1994.575864
std,4.946956,134.517281,32842.046199,6169.744172,3337.652630,0.493962,68.148608,4.918935,134.815493,33019.074419,...,29.892393,31270.631841,6114.320100,3030.723801,31406.825283,5736.911689,3355.845602,29398.677652,6348.562027,3689.507923
min,221.000000,20.000000,-127920.000000,0.000000,-29196.000000,0.000000,0.000000,219.000000,15.000000,-130406.000000,...,-231.000000,29.000000,0.000000,0.000000,21.000000,0.000000,0.000000,28.000000,0.000000,0.000000
25%,231.000000,125.000000,27662.000000,112.000000,-1327.000000,0.000000,4.000000,231.000000,128.000000,28253.000000,...,23.000000,28447.000000,163.000000,0.000000,29198.000000,97.000000,0.000000,28795.000000,120.000000,0.000000
50%,234.000000,199.000000,45173.000000,2265.000000,0.000000,0.000000,8.000000,235.000000,200.000000,45400.000000,...,27.000000,46130.000000,2377.000000,72.000000,46227.000000,1994.000000,140.000000,46522.000000,2106.000000,113.000000
75%,237.000000,300.000000,69113.000000,7441.000000,0.000000,1.000000,18.000000,238.000000,294.000000,67552.000000,...,31.000000,69649.000000,7434.000000,2314.000000,68176.000000,6904.000000,2554.000000,69531.000000,7275.000000,2702.000000
max,248.000000,1089.000000,252547.000000,43387.000000,16371.000000,1.000000,360.000000,248.000000,1240.000000,284702.000000,...,106.000000,360388.000000,58071.000000,34069.000000,382292.000000,53412.000000,38752.000000,312639.000000,70957.000000,40375.000000


### Part 1: Automating EDA - Using Sweetviz

In [ ]:
# Create analysis report
analyze_report = sv.analyze(data_new)
analyze_report.show_html()

In [ ]:
# Create a Train and Test Data Comparison - Differences and Similarities
compare = sv.compare([df_train, "Training Data"], [df_test, "Test Data"], "W_L1")
compare.show_html()

In [12]:
# Create a Comparison - Particular column

intra_com = sv.compare_intra(data_new[["date", "hour", "substation", "App SW", 
                        "V_L1", "I_L1", "W_L1", "QL_L1", "QC_L1","cos_L1", "angle_L1",
                        "V_L2", "I_L2", "W_L2", "QL_L2", "QC_L2","cos_L2", "angle_L2",
                        "V_L3", "I_L3", "W_L3", "QL_L3", "QC_L3","cos_L3", "angle_L3",
                        "temp_amb"]], data_new["substation"] == "S242", ["S201", "S2274", "S242", "S286", "S287", "S406", "S480", "S499", "S531", "S612", "S68638", "S7116", "S733", "S740", "S744", "S76020", "S813", "S820", "S850", "S868"])
intra_com.show_html(filepath='Compare_Intra.html', open_browser=True, layout='widescreen', scale=None)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:04 -> (00:00 left)
Report Compare_Intra.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
# Display the info on the output cell, not the browser
# report.show_notebook(w=None, h=None, scale=None,layout='widescreen',filepath=None)

### Part 2: Automating EDA - Using Autoviz

In [8]:
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()

df1 = AV.AutoViz(filename="", dfte=data_new, chart_format="svg", max_rows_analyzed=10000, max_cols_analyzed=15,)

Shape of your Data Set: (164433, 38)
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    Number of Numeric Columns =  28
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  2
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  3
    Number of Discrete String Columns =  2
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  3
    38 Predictors classified...
        This does not include the Target column(s)
        5 variables removed since they were ID or low-information variables
Since Number of Rows in data 164433 exceeds maximum, randomly sampling 50000 rows for EDA...
20 numeric variables in data exceeds limit, taking top 20 variables
Number of All Scatter Plots = 210


### Part 3: Automating EDA - Using Lux

In [ ]:
data_new

In [ ]:
data_new.intent = ["I_L1", "W_L1"]
data_new

In [ ]:
interestingVis = data_new.exported
interestingVis

In [ ]:
interestingVis[0]

In [ ]:
X.recommendation["Enhance"]

In [ ]:
print(interestingVis[0].to_matplotlib())

### Part 4: Filter useful info

In [ ]:
data = data.drop(["aminus_L1", "RminusL_L1", "RplusC_L1", 
                  "aminus_L2", "RminusL_L2", "RplusC_L2",
                  "aminus_L3", "RminusL_L3", "RplusC_L3"], axis=1)


In [ ]:
# Randomly shuffle a dataframe
data.reindex(np.random.permutation(data.index))